In [1]:
%matplotlib agg
# Avoid redundent output

from necessary_condition import *
from chemical_network_examples import examples, open_reference
from filedialog import SaveButton
from plots import *

from collections import OrderedDict

from ipywidgets import interact, interactive
from ipywidgets import Button, Checkbox, FloatSlider, Image, Label, Output, Text
from ipywidgets import Accordion, Box, HBox, VBox, Tab


class ReactionWidget(HBox):
    def __init__(self, parent, index):
        super(ReactionWidget, self).__init__()
        
        self.parent = parent
        self._index = index
        self.named = False
        self.updating = False
        
        namedbutton = Button(icon="unlock",
                             disabled=True,
                             style=dict(button_color="white"),
                             layout=dict(width="35px"),
                             tooltip=("The name of this reaction is generated automatically "
                                      "and may be udpated when another reaction is deleted"))
        
        name = Text(value="R{}".format(index), layout=dict(width="80px"))
        name.observe(self.change_name, names="value")
        
        reactants = Text(placeholder="Reactants")
        products = Text(placeholder="Products")
        
        arrow = Image(format="png", layout=dict(height="15px", margin="8px 25px 0px 25px"))
        
        reversible = Checkbox(description="Reversible", indent=False, layout=dict(width="100px"))
        reversible.observe(self.change_arrow, names="value")
        
        deletebutton = Button(icon="times",
                              tooltip="Delete this reaction",
                              layout=dict(width="40px"))
        deletebutton.on_click(lambda button: parent.delete_reaction(self))
        
        self.widgets = OrderedDict(namedbutton=namedbutton,
                                   name=name,
                                   reactants=reactants,
                                   arrow=arrow,
                                   products=products,
                                   reversible=reversible,
                                   deletebutton=deletebutton)
        
        self.children = tuple(self.widgets.values())
        self.layout = layout=dict(width="90%")
        
        self.change_arrow()
    
    # Using widget as attribute accesses their values
    def __getattr__(self, attr):
        if attr in self.widgets.keys():
            return self.widgets[attr].value
        else:
            raise AttributeError
    
    def __setattr__(self, attr, value):
        if attr in self.widgets.keys():
            self.widgets[attr].value = value
        else:
            super(ReactionWidget, self).__setattr__(attr, value)
    
    @property
    def index(self):
        return self._index
    
    @index.setter
    def index(self, k):
        self._index = k
        if not self.named:
            self.name = "R{}".format(k)
    
    def change_arrow(self, change=dict(new=False)):
        isrev = change["new"]
        if isrev:
            file = "reversiblearrow.png"
        else:
            file = "rightarrow.png"
        
        self.widgets["arrow"].set_value_from_file(file)
        
    def change_name(self, change):
        if not self.updating:
            self.named = True
            self.widgets["namedbutton"].icon = "lock"
            self.widgets["namedbutton"].tooltip = "The name of this reaction is locked and won't change"
        self.name = change["new"]

    
# TODO fix tab title
# TODO remove everything if last tab is closed
# TODO nicer result output (maybe using HTML) and choose what to display
# TODO additional accordion will fuller info ?
class ResultWidget(VBox):
    def __init__(self, parent, result, influence_graph):
        super(ResultWidget, self).__init__()
        self.parent = parent
        
        if len(influence_graph.nodes) < 2:
            error_out = Output()
            with error_out:
                raise ValueError("The reaction network should have at least two species.")
            self.children = [error_out]
            return
            
            
        res_out = Output()
        
        with res_out:
            if result["possible_multistability"]:
                print("Multistability can not be excluded for this system.")
                print("Admissible hooping (det = {}):".format(result["det"]))
                for c in result["hooping"]:
                    print("  ", c)
                    
                print("Corresponding reactions:")
                for p in result["path"]:
                    print("  ", p)
            else:
                print("Multistability is impossible for this system.")
        
        self.mutable_influence_graph = plot_influence_graph(influence_graph)
        if result["possible_multistability"]:
            self.mutable_influence_graph.highlight_hooping(result["hooping"], result["path"])
            
        graph_out = self.influence_graph_output()
        self.graphbox = Box([graph_out])
        
        
        sepslider = FloatSlider(
            value=0.1,
            min=0,
            max=0.3,
            step=0.001,
            description='Multi-edges separation',
            readout=False,
            continuous_update=False,
            style=dict(description_width="initial"),
            layout=dict(justify_content="space-around"))
        sepslider.observe(self.change_sep, names="value")
        
        closebutton = Button(description="Close",
                             icon="window-close",
                             tooltip="Close this result tab",
                             layout=dict(width="20%", height="30px"))
        closebutton.on_click(lambda button: self.parent.close_result(self))
        
        savebutton = SaveButton()
        savebutton.save_function = lambda path: self.mutable_influence_graph.figure.savefig(path, bbox="tight")
        
        self.children = [res_out, self.graphbox, sepslider, savebutton, closebutton]

    def change_sep(self, change):
        self.mutable_influence_graph.update_sep(change["new"])
        self.graphbox.children = (self.influence_graph_output(),)
    
    def influence_graph_output(self):
        out = Output()
        
        with out:
            display(self.mutable_influence_graph.figure)
            
        return out
    
class ExampleWidget(HBox):
    def __init__(self, parent, ex):
        super(ExampleWidget, self).__init__()
        self.parent = parent
        examplebutton = Button(description=ex["name"],
                               layout=dict(description_width="initial", 
                                      width="80%"))
        examplebutton.on_click(parent.fill_function(ex))

        refbutton = Button(description="Ref.",
                           icon="external-link",
                           tooltip=ex["ref"],
                           layout=dict(width="80px"))
        refbutton.on_click(self.reflink_function(ex))

        self.children = [examplebutton, refbutton]
        self.layout=dict(width="70%")
        
    # Needed to keep everything local and avoid weird callback behavior
    @staticmethod
    def reflink_function(ex):
        return lambda button: open_reference(ex)
    

class MainWidget(VBox):
    def __init__(self):
        super(MainWidget, self).__init__()
        example_accordion = Accordion
        
        example_box = Box([ExampleWidget(self, ex) for ex in examples],
                          layout=dict(flex_flow="column wrap",
                                      justify_content="space-around"))
        example_accordion = Accordion(children=[example_box])
        example_accordion.selected_index = None
        example_accordion.set_title(0, "Examples")
            
            
        addbutton = Button(icon="plus", tooltip="Add a reaction", layout=dict(width="60px", height="auto"))
        addbutton.on_click(self.new_reaction)

        runbutton = Button(description="Check for multistability",
                           icon="play",
                           tooltip="Start the checking process",
                           layout=dict(width="40%", height="40px"))
        runbutton.on_click(self.run_analysis)
        
        resetbutton = Button(description="Reset reactions",
                             icon="undo",
                             tooltip="Reset reactions to their original empty state",
                             layout=dict(height="auto"))
        resetbutton.on_click(self.reset)
        
        self.reactions = []
        self.reactions_box = VBox()
        self.resbox = Box()
        self.new_reaction()
        
        reactions_box = Box([self.reactions_box, addbutton])
        self.children = [example_accordion,
                         reactions_box,
                         Box([runbutton, resetbutton], layout=dict(justify_content="space-around")),
                         self.resbox]
        
        self.result_tab = None
        self.restabs = []
        
    @property
    def reaction_data(self):
        return [dict(
            before=r.reactants,
            after=r.products,
            reversible=r.reversible,
            name=r.name,
            named=r.named
            ) for k, r in enumerate(self.reactions) if (r.reactants != "" and r.products != "")]
    
    def close_result(self, res):
        self.restabs.remove(res)
        self.result_tab.children = tuple([r.wrapped_widget for r in self.restabs])
        
    def delete_reaction(self, R):
        self.reactions.remove(R)
        self.reactions_box.children = tuple(self.reactions)
        
        for k, r in enumerate(self.reactions):
            r.updating = True
            r.index = k
            r.updating = False
    
    def delete_all_reactions(self):
        self.reactions_box.children = tuple()
        self.reactions = []
    
    def fill_with_example(self, ex):
        self.delete_all_reactions()
        reaction_data = split_reactions(ex["network"])
        
        for reaction in reaction_data:
            reaction_widget = self.new_reaction(update=False)
            reaction_widget.reactants = reaction["before"]
            reaction_widget.products = reaction["after"]
            reaction_widget.reversible = reaction["reversible"]
            reaction_widget.name = reaction["name"]
        
        self.update()
    
    # Needed to keep everything local and avoid weird callback behavior
    def fill_function(self, ex):
        return lambda button: self.fill_with_example(ex)
            
    def new_reaction(self, button=None, update=True):
        reaction_widget = ReactionWidget(self, len(self.reactions))
        self.reactions.append(reaction_widget)
        
        if update:
            self.update()
        
        return reaction_widget
    
    def reset(self, button=None):
        self.delete_all_reactions()
        self.new_reaction()
    
    def run_analysis(self, button=None):
        if self.result_tab is None:
            self.result_tab = Tab()
            self.resbox.children = (self.result_tab, )
        
        result, influence_graph = test_multistability(self.reaction_data)
        
        res = ResultWidget(self, result, influence_graph)
        
        index = len(self.restabs)
        self.restabs.append(res)
        self.result_tab.children = tuple(self.restabs)
        self.result_tab.selected_index = index
        self.result_tab.set_title(index, "Result {}".format(index + 1))
    
    def update(self):
        self.reactions_box.children = tuple(self.reactions)
        
        
mw = MainWidget()
mw
# TODO better classified examples and more examples
# TODO use example in test suite
# TOFIX missing highlight in aejknp

MainWidget(children=(Accordion(children=(Box(children=(ExampleWidget(children=(Button(description='Double nega…